In [1]:
from fastai.imports import *
np.set_printoptions(linewidth=130)

import os
iskaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')

if iskaggle: path = Path('../input/heart-failure-prediction')
else:
    import zipfile,kaggle
    path = Path('heart-failure-prediction')
    kaggle.api.competition_download_cli(str(path))
    zipfile.ZipFile(f'{path}.zip').extractall(path)